In [10]:
from google.colab import files
uploaded = files.upload()


Saving orders.csv to orders.csv


In [11]:
from google.colab import files
uploaded = files.upload()


Saving returns.csv to returns (1).csv


In [12]:
import pandas as pd

orders = pd.read_csv('orders.csv')
returns = pd.read_csv('returns.csv')

# Clean and merge
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
returns['ReturnDate'] = pd.to_datetime(returns['ReturnDate'])

df = pd.merge(orders, returns, on='OrderID', how='left')
df['IsReturned'] = df['ReturnDate'].notnull().astype(int)


In [13]:
df.head()

,OrderID,ProductID,ProductName,Category,Supplier,Price,Region,OrderDate,ReturnDate,IsReturned
0,1,1006,Phone,Books,SupplierB,197,East,2023-01-01,NaT,0
1,2,1009,Shirt,Books,SupplierC,554,South,2023-01-02,NaT,0
2,3,1010,Book,Books,SupplierC,776,East,2023-01-03,2023-06-08,1
3,4,1006,Shoes,Clothing,SupplierA,895,South,2023-01-04,NaT,0
4,5,1001,Shirt,Books,SupplierC,768,East,2023-01-05,NaT,0


In [14]:
category_return_rate = df.groupby('Category')['IsReturned'].mean().reset_index()
supplier_return_rate = df.groupby('Supplier')['IsReturned'].mean().reset_index()


In [15]:
print(category_return_rate)
print(supplier_return_rate)

      Category  IsReturned
0        Books    0.210526
1     Clothing    0.111111
2  Electronics    0.257143
    Supplier  IsReturned
0  SupplierA    0.212121
1  SupplierB    0.178571
2  SupplierC    0.205128


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report


df_model = pd.get_dummies(df[['Category', 'Supplier', 'Price', 'Region', 'IsReturned']], drop_first=True)

X = df_model.drop('IsReturned', axis=1)
y = df_model['IsReturned']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


df['Return_Prob'] = model.predict_proba(X)[:, 1]


              precision    recall  f1-score   support

           0       0.85      1.00      0.92        17
           1       0.00      0.00      0.00         3

    accuracy                           0.85        20
   macro avg       0.42      0.50      0.46        20
weighted avg       0.72      0.85      0.78        20



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
high_risk_products = df[df['Return_Prob'] > 0.7]
high_risk_products[['ProductID', 'ProductName', 'Return_Prob']].to_csv('high_risk_products.csv', index=False)


In [18]:
print(high_risk_products.head())

Empty DataFrame
Columns: [OrderID, ProductID, ProductName, Category, Supplier, Price, Region, OrderDate, ReturnDate, IsReturned, Return_Prob]
Index: []


In [19]:

df.to_csv('cleaned_data.csv', index=False)
files.download('cleaned_data.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>